In [4]:
!pip list

Package                   Version
------------------------- -----------
aiohttp                   3.8.5
aiosignal                 1.3.1
annotated-types           0.5.0
anyio                     3.7.1
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.2.3
assemblyai                0.15.1
asttokens                 2.2.1
async-lru                 2.0.4
async-timeout             4.0.3
attrs                     23.1.0
autochain                 0.0.5
Babel                     2.12.1
backcall                  0.2.0
backoff                   2.2.1
bcrypt                    4.0.1
beautifulsoup4            4.12.2
bleach                    6.0.0
boltons                   23.0.0
brotlipy                  0.7.0
certifi                   2023.7.22
cffi                      1.15.1
charset-normalizer        2.0.4
chroma-hnswlib            0.7.2
chromadb                  0.4.8
click                     8.1.7
colorama                  0.4.6
coloredlogs         

In [ ]:
import 